# Practices(1): 정규 표현 메타 기호
--- 
All rights reserved, 2021-2024 by *Youn-Sik Hong*. 수업 목적으로만 활용 가능.

# Assignment
- 이 handout sheet에는 마지막에 2개의 문제가 있습니다. quiz가 아니라 문제입니다.
    - 비어있는 셀에 문제에 대한 정답 코드를 작성하고 실행 결과를 보여야 합니다.
- 모든 셀의 실행 결과가 포함되어 있어야 합니다. 
    - 즉, 모든 셀에는 셀 번호가 있어야 합니다.

In [1]:
import re

정규표현 기본기(메타기호)를 제대로 익혔는지 복습해 볼까요?

In [2]:
def text_search(patterns, text):
    m = re.search(patterns, text)
    if m:
        print(f'found {m.group(0)}, len={len(m.group(0))}') 
    else:
        print('Not found!')

## ? : zero or one occurrence
- 메타기호 \? 는 패턴이 없거나 한 번 발생(zero or one occurrence)을 의미합니다. 
---
- 패턴이 없다는 것은 생략, 즉 빈 문자열(empty string)을 의미합니다. 
    - empty string은 Greek letter인 $\epsilon$ (또는 $\lambda$)로 표기합니다.
    - empty string은 길이가 0인 문자열입니다. 
    - 문자열 연산에서 empty string은 곱셈의 1과 같습니다. 
        - 곱셈에서 1을 곱해도 전혀 값이 변하지 않는 것과 같습니다. 

In [3]:
# ab? 패턴에 해당하는 문자열: a, ab
#text_search(r"ab?", "ac")
text_search("ab?", "ac")  
text_search("ab?", "abc")
text_search("ab?", "bbc")

found a, len=1
found ab, len=2
Not found!


- 괄호로 묶으면 어떻게 될까요? 
    - **ab?** 패턴에 해당하는 문자열이 [a, ab]라는 것은 **a(b?)** 와 같습니다.
    - 그러면 **(ab)?** 패턴에 해당하는 문자열은 [$\epsilon$, ab] 이겠죠. 
    - 실행하면서 확인해 볼까요? **(ab)?** 는 아무 것도 출력을 하지 않네요..뭔가 이상하군요.

In [4]:
text_search(r"a(b)?", "ac")
text_search(r"(ab)?", "ac")

found a, len=1
found , len=0


- **text_search** 메소드를 조금 수정하면 금방 그 이유를 알 수 있습니다. 
    - 괄호를 사용했기 때문에 subgroup을 확인할 수 있습니다.
    - **match** 객체가 *None*은 아닙니다. 
        - m.group(1)이 None 인 것은 empty string이기 때문입니다.
        - empty string이니 보일 리가 없죠...

In [5]:
m = re.search("(ab)?", "ac")
if m:
    print(f'match is not null... [{m.group(1)}]') 
else:
    print('Not found!')

match is not null... [None]


## * : zero or more occurrences
- 메타기호 \* (star로 읽음)는 패턴이 아예 나타나지 않거나(생략) 여러 번 나타남을 의미합니다. 
- 패턴이 생략될 수 있다는 점이 \+와 차이점입니다.

In [6]:
# ab* = a(b)* 패턴에 해당하는 문자열: a, ab, abb, abbb, abbb....
# text_search(r"ab*", "ac") # r은 생략 가능
text_search("ab*", "ac")  
text_search("ab*", "abc")
text_search("ab*", "abbbbbc")
text_search("ab*", "cbc")

found a, len=1
found ab, len=2
found abbbbb, len=6
Not found!


## +: one or more occurrences
- 메타기호 \+ 는 최소 한 번 나타나거나 여러 번 나타남을 의미합니다. 
- 최소 한 번은 패턴이 나타나야 합니다.

In [7]:
# ab+ = a(b)+ 패턴에 해당하는 문자열: {ab, abb, abbb, abbbb, ....}
#text_search(r"ab+", "ac") # r은 생략 가능
text_search("ab+", "ac") 
text_search("ab+", "abc")
text_search("ab+", "abbc")

Not found!
found ab, len=2
found abb, len=3


- 메타기호 \{ \} 를 사용하여 패턴의 발생 횟수 또는 범위를 지정할 수 있습니다. 
    - \{3, 5\}는 패턴이 3번, 4번, 또는 5번 발생할 수 있다는 뜻입니다.
    - 또, 최소 3번은 발생해야 한다는 뜻입니다.

In [8]:
text_search("ab{2}", "abbc") # abb
text_search("ab{3,5}", "aabbbbbbc")  # abbb, abbbb, abbbbb
text_search("ab{3,5}", "aabbc") # abbb, abbbb, abbbbb
text_search("(ab){3,5}", "aabbc") #ababab. abababab, ababababab
text_search("a(b{3,5})", "abbbc") #abbb, abbbb, abbbbb

found abb, len=3
found abbbbb, len=6
Not found!
Not found!
found abbb, len=4


- 여기서 잠깐! 최소 3번 이상 발생은 어떻게 지정할까요?
- 아래 예제 문자열에서 b 의 개수는 모두 6개입니다.

In [9]:
text_search("ab{3,5}", "aabbbbbbbc") # 3,4,5번 발생
text_search("ab{3,}", "aabbbbbbbc")  # 3번 이상 발생한 패턴 중에 가장 길이가 긴 패턴을 찾음

found abbbbb, len=6
found abbbbbbb, len=8


## ^, $
- 메타기호 \^(caret)은 텍스트의 맨 처음을 나타내는 기호입니다. 
- 메타기호 \$(dollar)는 텍스트의 맨 마지막을 나타내는 기호입니다.
- 아래 예제는 a로 시작해서 c로 끝나는 문자열을 찾습니다.

In [10]:
text_search("^a.*c$", "abfc", )  # a...c
text_search("^a.*c$", "abfck", )  # a...c
text_search("^a.*k$", "abfck", )  # a...k

found abfc, len=4
Not found!
found abfck, len=5


### 문자: \w, \W
- \w : Any **alphanumeric** character (equivalent to [a-zA-Z0-9_]). 
- \W : Any **non-alphanumeric** character (equivalent to  [^a-zA-Z0-9_])

- text_search()는 \^ 이 있기 때문에 영문자+숫자로 이루어진 단어를 찾습니다.
- 첫 번째 findall() 역시 \^ 을 사용했기 때문에 조건에 맞는 단어는 하나 뿐입니다.
- 두 번째 findall()은 \^ 이 없기 때문에 영문자+숫자로 이루어진 모든 단어를 찾습니다.
    - 구둣점 기호(, !)와 공백 문자(space)는 제외됩니다.
- 세 번째 findall()은 대문자 W를 사용했기 때문에 구둣점 기호와 공백 문자만 찾습니다.

In [11]:
str = "Tuffy eats pie, Loki eats peas!"
text_search("^\w+", str)

print(re.findall('^\w+', str))
print(re.findall('\w+', str))
print(re.findall('\W+', str))

found Tuffy, len=5
['Tuffy']
['Tuffy', 'eats', 'pie', 'Loki', 'eats', 'peas']
[' ', ' ', ', ', ' ', ' ', '!']


### 공백 문자: \s, \S
- \s : Any whitespace character (equivalent to [ \t\n\r\f\v]).
    - \t(tab), \n(newline), \r(carriage return) 등은 **escape 문자** 입니다.
        - Tab 키, Enter 키, Space 키에 해당하는 문자를 가리킵니다.
    - [ ]기호(brackets) 의미를 정확히 이해하세요. 
        - [ ]안에 속한 기호 중 **하나**를 가리킵니다. 
    - 아래 예제에서 여러 개의 whitespace 문자를 찾기 때문에, 
        - \s대신 \s+와 같이 메타기호 +를 추가했습니다.
---    
- \S : Any non-whitespace character (equivalent to [^ \t\n\r\f\v])
    - 눈치빠른 학생은 지금쯤 ^(caret) 기호 의미를 파악했을 겁니다. 
    - 그리고, 소문자 메타기호와 대문자 메타기호는 정반대의 패턴을 정의한다는 것도요!!! 

In [12]:
str = "Tuffy eats pie, Loki eats peas!"
print('non-whitespace chars=', end='')
text_search(r"\S+", str)

print('whitespace chars=', end='')
text_search(r"\s+", str)

non-whitespace chars=found Tuffy, len=5
whitespace chars=found  , len=1


- whitespace chars는 아무 것도 출력되지 않았어요.
- 아래 코드를 실행시켜 보세요.

In [13]:
s1 = re.findall(r'\s+', str)
print(f'찾은 토큰 개수 = {len(s1)}, whitespace chars = {s1}')
s2 = re.findall(r'\S+', str)
print(f'찾은 토큰 개수 = {len(s2)}, non-whitespace chars = {s2}')

찾은 토큰 개수 = 5, whitespace chars = [' ', ' ', ' ', ' ', ' ']
찾은 토큰 개수 = 6, non-whitespace chars = ['Tuffy', 'eats', 'pie,', 'Loki', 'eats', 'peas!']


한 가지 아니면 2개 메타 기호를 사용한 정규표현 예제가 지루해졌죠. 이제 여러 개 메타 기호를 섞어 쓴 예제를 살펴봅시다.
- \w+와 \S+ 차이점에 주목하기 바랍니다.
- \S+는 구둣점도 포함해서 token을 찾습니다. 

In [14]:
str = "Tuffy? eats^ pie#, Loki+ eats~ peas!"

print(re.findall("\S+", str))
print(re.findall("\S+$", str))
print(re.findall("^\S+", str))

['Tuffy?', 'eats^', 'pie#,', 'Loki+', 'eats~', 'peas!']
['peas!']
['Tuffy?']


- 설명할까 그냥 넘어갈까 고민한 메타 표현이 \b와 \B입니다. 조금 헷갈릴 수 있기 때문입니다. 예제를 잘 보고 이해하기 바랍니다.
- '\B'는 단어와 단어 사이에 있는 space나 구둣점('.')과 매칭합니다.
    - 예: "A" -> 0, "AB" -> 1, "ABC" -> 2, "ABC." -> 3... 등

- str = "ABC DE"일 때 \B 패턴은 3곳을 찾았습니다. 왜 그럴까요? 
- 주석 처리해 놓은 str을 바꿔가며 결과를 확인하세요.

In [18]:
str = "AB"
#str = "ABC"
#str = "ABC DE"
for match in re.finditer(r'\B', str):
    s = match.start()
    e = match.end()
    print('Found "%s" at %d:%d' % (str[s:e], s, e))

Found "" at 1:1


- 패턴'\Bu\B'는 문자와 문자 사이에 'u'가 포함되어 있는가 입니다.
- 주석 처리해 놓은 str을 바꿔가며 결과를 확인하세요.

In [22]:
str = "Tommy eats pie, Loki eats peas!"
#str = "Tuffy eats pie, Loki eats peas!"
text_search(r"\Bu\B", str)

Not found!


- '\b'가 '\B'보다 사용하기 편합니다. 
    - 왜냐하면 '\B'는 글자와 글자 사이까지 속속들이 찾는 반면, 
    - '\b'는 단어(word)의 시작과 끝에서만 찾기 때문이죠. 
    - '\b'는 거의 대부분 '\w'와 함께 사용하며, '\w'에 해당하는 단어의 경계(boundary)를 정의할 목적으로 사용합니다. 
- 매뉴얼에서 설명한 예가 이를 잘 설명하고 있습니다. 
    - r'\bfoo\b' 는 'foo', 'foo.', '(foo)', 'bar foo baz' 와 매칭되지만, 'foobar', 'foo3'는 매칭되지 않습니다. 
        - 즉 단어 'foo'만을 찾을 목적으로 사용하라는 겁니다. 
        - 'foobar'나 'foo3'는 그 자체로 한 단어이기 때문에, 'foo'와는 전혀 다르죠.

위에 지루한 설명은, 아래 예로 금방 확인할 수 있습니다. 주어진 문장에서 공백과 공백 사이에 있는 단어 중에 길이가 5인 단어를 찾아보자구요.

In [23]:
text = "Love is like a sweet whisper, autumn winds!"
text_search(r"\b\w{5}\b", text)
print(re.findall(r"\b\w{5}\b", text))

found sweet, len=5
['sweet', 'winds']


자! 이제부터 도전 과제로 넘어갑니다. 자신의 skill이 어느 정도에 도달했는지 테스트해 볼 수 있는 기회입니다.

### 문제 1(difficulty=low)
아래 단어에서 모음은 모두 몇 개일까요?

In [176]:
word = 'supercalifragilisticexpialidocious'

In [208]:
v = re.findall(r"\B[aeiou]\B", word)
print(v)
print(len(v))

['u', 'e', 'a', 'i', 'a', 'i', 'i', 'i', 'e', 'i', 'a', 'i', 'o', 'i', 'o', 'u']
16


### 문제 2(difficulty=medium)
아래 url에서 년도(year), 월(month), 일(day)을 찾아 보세요.

In [93]:
url= "http://www.telegraph.co.uk/formula-1/2017/10/28/"\
     "mexican-grand-prix-2017-time-does-start-tv-channel-odds-lewis1/2017/05/12/"
url

'http://www.telegraph.co.uk/formula-1/2017/10/28/mexican-grand-prix-2017-time-does-start-tv-channel-odds-lewis1/2017/05/12/'

In [94]:
def extract_string(pattern, str):
    for match in re.finditer(pattern, str):
        s = match.start()
        e = match.end()
        print('Found "%s" at %d:%d' % (str[s:e], s, e))

- 먼저 년도(year)부터 찾아봅시다. 년도가 17이 아니라 2017, 이렇게 표현되어 있군요. 이게 년도의 패턴이겠죠. 
- 또 finditer() 메소드를 자주 사용하니까 함수로 만들어 놓는게 좋겠죠.

In [191]:
re_year = r'\d{4}'
extract_string(re_year, url)

Found "2017" at 37:41
Found "2017" at 67:71
Found "2017" at 111:115


In [205]:
# 문제 2: 당신의 능력을 보여주세요.
re_month_day = r'/\d{2}/\d{2}/'
extract_string(re_month_day, url)

Found "/10/28/" at 41:48
Found "/05/12/" at 115:122


- Great! 여기까지 온 당신은 장애물을 모두 돌파할 수 있는 모든 준비를 다 갖췄습니다. 
- 이제 Practices(2)에 준비된 difficulty level=high에 도전해 봅시다.